In [3]:
lii = "left_instance_id"
rii = "right_instance_id"
lbl = "label"


In [4]:
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
gold = pd.read_csv("../data/sigmod/Y4.csv")
# gold = pd.read_csv("Y3_1only.csv")
# gold = pd.read_csv("Y4_1only.csv")
x = pd.read_csv("../data/sigmod/X4.csv")
my_y_filename = "../src/dedupe/cleaning.csv"
final_save_name = "y4_with_real.csv"

In [7]:
mine = pd.read_csv("../src/dedupe/output.csv")

In [8]:
mine.head()

,left_instance_id,right_instance_id
0,altosight.com//0,altosight.com//11024
1,altosight.com//0,altosight.com//11026
2,altosight.com//0,altosight.com//240
3,altosight.com//0,altosight.com//2534
4,altosight.com//0,altosight.com//3076


In [9]:
gold_dict = {}
for index, row in tqdm(gold.iterrows()):
    l, r, label = row[lii], row[rii], row[lbl]
    if label == 0:
        continue
        
    s = l + "|" + r
    gold_dict[s] = 1
#     if l not in gold_dict:
#         gold_dict[l] = {}
#     if r not in gold_dict[l]:
#         gold_dict[l][r] = 1

0it [00:00, ?it/s]

In [10]:
mine_dict = {}
for index, row in tqdm(mine.iterrows()):
    l, r = row[lii], row[rii]
    s = l + "|" + r
    mine_dict[s] = 1

0it [00:00, ?it/s]

In [11]:
len(gold)

348195

In [12]:
len(gold_dict)

4082

In [13]:
len(mine_dict)

7240

In [14]:
false_positive = []
for index, row in tqdm(mine.iterrows()):
    l, r = row[lii], row[rii]
    str1 = l + "|" + r
    str2 = r + "|" + l
    if str1 not in gold_dict and str2 not in gold_dict:
        false_positive.append((l,r))

0it [00:00, ?it/s]

In [15]:
len(false_positive)

3720

In [16]:
false_negative = []
for index, row in tqdm(gold.iterrows()):
    l, r, label = row[lii], row[rii], row[lbl]
    if label == 0:
        continue
        
    str1 = l + "|" + r
    str2 = r + "|" + l
    if str1 not in mine_dict and str2 not in mine_dict:
        false_negative.append((l,r))

0it [00:00, ?it/s]

In [17]:
len(false_negative)

562

In [80]:
# 问题：这里面的是怎么算出来的？？？


In [18]:
tp = len(gold_dict) - len(false_negative)
tp2 = len(mine_dict) - len(false_positive)
print(tp, tp2)

fp = len(false_positive)
fn = len(false_negative)

precision = tp / (tp+fp)
recall = tp / (tp+fn)
f1 = 2 * (precision * recall) / (precision + recall)

print("tp",tp)
print("fp",fp)
print("fn",fn)
print("precision",precision)
print("recall", recall)
print("F1",f1)

3520 3520
tp 3520
fp 3720
fn 562
precision 0.4861878453038674
recall 0.8623223909848113
F1 0.6217982688570923


In [19]:
len(x)

835

In [274]:
343 * 342 / 2

58653.0

In [275]:
mine.head()

,left_instance_id,right_instance_id
0,altosight.com//0,altosight.com//2534
1,altosight.com//0,altosight.com//4054
2,altosight.com//0,altosight.com//6380
3,altosight.com//0,altosight.com//760
4,altosight.com//2534,altosight.com//4054


In [276]:
# x_dict = {}
# for idx, row in tqdm(x.iterrows()):
#     obj = {"title": row["title"]}
#     x_dict[row["instance_id"]] = obj

In [277]:
# debug = pd.read_csv("debug.csv")

In [28]:
import pickle
with open("compute_x4.model", "rb") as f:
    debug = pickle.load(f)

In [29]:
# debug_dict = {}
# for idx, row in tqdm(debug.iterrows()):
#     obj = {"line": row["line"], "model": row["model"], "brand": row["brand"], "title": row["title"]}
#     debug_dict[row["instance_id"]] = obj

In [30]:
debug.head()

AttributeError: 'DataModel' object has no attribute 'head'

In [280]:
debug_all_dict = debug.to_dict("records")

In [281]:
debug_all_list = debug_all_dict

In [282]:
debug_all_dict = {}
for item in debug_all_list:
    debug_all_dict[item["instance_id"]] = item

In [283]:
x_all_dict = x.to_dict('records')

In [284]:
x_all_dict[0]

{'name': 'Lexar 32GB 1400x 210MB/s professional XQD memóriakártya',
 'price': 31990.0,
 'brand': 'LEXAR',
 'size': '32 GB',
 'instance_id': 'altosight.com//0'}

In [285]:
x_all_list= x_all_dict

In [286]:
x_all_dict = {}
for item in x_all_list:
    x_all_dict[item["instance_id"]] = item

In [22]:
from collections import Counter

In [23]:
all_fp_comb = []
all_fp_counters = []
for (l,r) in false_positive:
    
    found_l, found_r = -1, -1
    for idx, cur_set in enumerate(all_fp_comb):
        if l in cur_set:
            found_l = idx
        if r in cur_set:
            found_r = idx
    
    if found_l == -1 and found_r == -1:
        new_set = set()
        new_set.add(l)
        new_set.add(r)
        all_fp_comb.append(new_set)
        all_fp_counters.append(Counter([l,r]))
    elif found_l != -1 and found_r == -1:
        all_fp_comb[found_l].add(r)
        all_fp_counters[found_l].update([l,r])
    elif found_l == -1 and found_r != -1:
        all_fp_comb[found_r].add(l)
        all_fp_counters[found_r].update([l,r])
        
    else:
        if found_l == found_r:
            continue
        else:
            print(l, r, found_l, found_r)
            raise Exception("wtf?")
        

In [24]:
len(all_fp_comb)

28

In [25]:
for idx, cluster in enumerate(all_fp_comb):
    print(idx, len(cluster), all_fp_counters[idx].most_common(2))

0 50 [('altosight.com//11319', 36), ('altosight.com//12629', 14)]
1 30 [('altosight.com//10040', 15), ('altosight.com//10450', 15)]
2 15 [('altosight.com//12307', 10), ('altosight.com//1204', 5)]
3 18 [('altosight.com//10449', 10), ('altosight.com//10453', 8)]
4 10 [('altosight.com//2283', 6), ('altosight.com//12488', 4)]
5 14 [('altosight.com//10677', 10), ('altosight.com//10448', 4)]
6 5 [('altosight.com//3156', 4), ('altosight.com//10480', 1)]
7 13 [('altosight.com//10451', 9), ('altosight.com//10447', 4)]
8 4 [('altosight.com//7300', 3), ('altosight.com//2228', 1)]
9 2 [('altosight.com//1449', 1), ('altosight.com//5550', 1)]
10 31 [('altosight.com//1198', 19), ('altosight.com//13252', 12)]
11 37 [('altosight.com//11165', 27), ('altosight.com//11316', 10)]
12 16 [('altosight.com//10501', 14), ('altosight.com//10532', 2)]
13 23 [('altosight.com//1399', 15), ('altosight.com//10508', 8)]
14 2 [('altosight.com//13177', 1), ('altosight.com//13179', 1)]
15 9 [('altosight.com//13208', 8), 

In [26]:
fp_dict = []
for (l,r) in false_positive:
    obj = {}
    obj["left"] = l
    obj["right"] = r
    
    found_l = -1
    found_r = -1
    for idx, cur_set in enumerate(all_fp_comb):
        if l in cur_set:
            found_l = idx
        if r in cur_set:
            found_r = idx
    obj["left_cluster_id"] = found_l
    obj["right_cluster_id"] = found_r
    for k,v in debug_all_dict[l].items():
        obj["left_" + k] = v
    for k,v in debug_all_dict[l].items():
        obj["left_all_" + k] = v
    for k,v in debug_all_dict[r].items():
        obj["right_" + k] = v
    for k,v in debug_all_dict[r].items():
        obj["right_all_" + k] = v
    fp_dict.append(obj)

NameError: name 'debug_all_dict' is not defined

In [20]:
fp_dict[1]

NameError: name 'fp_dict' is not defined

In [293]:
last_id = ""
split_points = []
for idx, item in enumerate(fp_dict):
    if item["left_cluster_id"] != last_id:
        split_points.append(idx)
        last_id = item["left_cluster_id"]

In [294]:
split_points

[0,
 25,
 46,
 67,
 84,
 88,
 91,
 101,
 131,
 141,
 154,
 182,
 197,
 217,
 225,
 351,
 411,
 628,
 673,
 679,
 864,
 865,
 874]

In [295]:
import json

In [296]:
with open("false_positive.json","w", encoding="utf8") as f:
    json.dump(fp_dict, f)

In [297]:
fn_dict = []
for (l,r) in false_negative:
    obj = {}
    obj["left"] = l
    obj["right"] = r
    for k,v in debug_all_dict[l].items():
        obj["left_" + k] = v
    for k,v in x_all_dict[l].items():
        obj["left_all_" + k] = v
    for k,v in debug_all_dict[r].items():
        obj["right_" + k] = v
    for k,v in x_all_dict[r].items():
        obj["right_all_" + k] = v
    fn_dict.append(obj)

In [298]:
from collections import Counter

In [299]:
Counter([item["left_brand"] for item in fp_dict])

Counter({'sandisk': 161,
         'sony': 82,
         'lexar': 23,
         'intenso': 28,
         'kingston': 43,
         'toshiba': 534,
         'samsung': 9})

In [300]:
Counter([item["left_line"] for item in fp_dict])

Counter({'extreme': 94,
         nan: 146,
         'ultra plus': 12,
         'ultra': 30,
         'dual drive': 2,
         'glide': 1,
         'performance': 1,
         'essential': 3,
         'jumpdrive': 22,
         'speed line': 22,
         'premium line': 6,
         'ultimate': 10,
         'datatraveler': 8,
         'exceria pro': 413,
         'exceria': 97,
         'pendrive': 6,
         'microvault': 7})

In [301]:
Counter([item["left_brand"] for item in fn_dict])

Counter({'lexar': 122,
         'sandisk': 151,
         'pny': 58,
         'sony': 84,
         'samsung': 27,
         'intenso': 14,
         'kingston': 53,
         'toshiba': 73,
         'transcend': 8})

In [302]:
Counter([item["left_line"] for item in fn_dict])

Counter({'xqd': 49,
         'dual drive': 6,
         'ultra performance': 9,
         'ultra plus': 10,
         'attache': 38,
         nan: 74,
         'premium': 16,
         'evo plus': 4,
         'basic line': 11,
         'datatraveler': 14,
         'jumpdrive': 46,
         'extreme': 63,
         'ultra': 34,
         'hyperx savage': 11,
         'exceria': 40,
         'cruzer': 4,
         'ultimate': 3,
         'pendrive': 36,
         'cruzer glide': 33,
         'galaxy': 23,
         'exceria pro': 21,
         'transmemory': 5,
         'microvault': 32,
         'platinum': 4,
         'hyperx': 2,
         'essential': 2})

In [303]:
fp_dict[0].keys()

dict_keys(['left', 'right', 'left_cluster_id', 'right_cluster_id', 'left_type', 'left_uhs', 'left_line', 'left_mymodel', 'left_micro', 'left_video_speed', 'left_usb_speed', 'left_name', 'left_price', 'left_brand', 'left_size', 'left_instance_id', 'left_all_type', 'left_all_uhs', 'left_all_line', 'left_all_mymodel', 'left_all_micro', 'left_all_video_speed', 'left_all_usb_speed', 'left_all_name', 'left_all_price', 'left_all_brand', 'left_all_size', 'left_all_instance_id', 'right_type', 'right_uhs', 'right_line', 'right_mymodel', 'right_micro', 'right_video_speed', 'right_usb_speed', 'right_name', 'right_price', 'right_brand', 'right_size', 'right_instance_id', 'right_all_type', 'right_all_uhs', 'right_all_line', 'right_all_mymodel', 'right_all_micro', 'right_all_video_speed', 'right_all_usb_speed', 'right_all_name', 'right_all_price', 'right_all_brand', 'right_all_size', 'right_all_instance_id'])

In [304]:
import csv

In [305]:
with open("fp.csv","w",encoding="utf8") as f:
    d = fp_dict
    csv_writer = csv.DictWriter(f, list(d[0].keys()))
    csv_writer.writeheader()
    csv_writer.writerows(d)

In [306]:
with open("fn.csv","w",encoding="utf8") as f:
    d = fn_dict
    csv_writer = csv.DictWriter(f, list(d[0].keys()))
    csv_writer.writeheader()
    csv_writer.writerows(d)

In [307]:
len(x)

835

In [308]:
# 尝试构建原始集合

In [309]:
all_real_comb = []
counter = 0
for index, row in tqdm(gold.iterrows()):
    l, r, label = row[lii], row[rii], row[lbl]
    if label == 0:
        continue
    
    counter += 1
    found_l, found_r = -1, -1
    for idx, cur_set in enumerate(all_real_comb):
        if l in cur_set:
            found_l = idx
        if r in cur_set:
            found_r = idx
    
    if found_l == -1 and found_r == -1:
        new_set = set()
        new_set.add(l)
        new_set.add(r)
        all_real_comb.append(new_set)
    elif found_l != -1 and found_r == -1:
        all_real_comb[found_l].add(r)
    elif found_l == -1 and found_r != -1:
        all_real_comb[found_r].add(l)     
    else:
        if found_l == found_r:
            continue
        else:
            lset = all_real_comb[found_l]
            rset = all_real_comb[found_r]
            all_real_comb[found_l]= lset.union(rset)
            all_real_comb[found_r]=set()
            
            print(l, r, found_l, found_r)
#             raise Exception("wtf?")
        

In [310]:
len([i for i in all_real_comb if len(i) == 0])

0

In [311]:
sum([len(i) for i in all_real_comb])

812

In [312]:
sum([(len(i)*len(i)-1)/2 for i in all_real_comb])

4440.0

In [313]:
check_dict = {}
for cur_set in all_real_comb:
    for item in cur_set:
        check_dict[item] = 1

In [314]:
for item in x_all_list:
    if item["instance_id"] not in check_dict:
        print(item["instance_id"])

altosight.com//1449
altosight.com//1521
altosight.com//1575
altosight.com//3101
altosight.com//3107
altosight.com//3111
altosight.com//3116
altosight.com//3117
altosight.com//3119
altosight.com//4909
altosight.com//5535
altosight.com//5668
altosight.com//5873
altosight.com//7815
altosight.com//7934
altosight.com//9058
altosight.com//12390
altosight.com//12391
altosight.com//12421
altosight.com//13208
altosight.com//13306
altosight.com//13369
altosight.com//13640


In [315]:
from itertools import permutations, combinations

In [316]:

for idx, cur_set in enumerate(all_real_comb):
    if len(cur_set) == 0:
        continue
    for cb in combinations(list(cur_set), 2):
        l, r = cb[0], cb[1]
        s1 = l + "|" + r
        s2 = r + "|" + l
        if s1 not in gold_dict and s2 not in gold_dict:
            print(idx, cb)

In [317]:
truth_li = []
truth_dict = {}
for idx, cur_set in enumerate(all_real_comb):
    if len(cur_set) == 0:
        continue
    for instance_id in cur_set:
        obj = debug_all_dict[instance_id]
        obj["real_cluster"] = idx
        truth_dict[instance_id] = idx
        truth_li.append(obj)

In [318]:
truth_li[0]

{'type': nan,
 'uhs': nan,
 'line': 'xqd',
 'mymodel': 'd32g',
 'micro': 'not micro',
 'video_speed': 'u1',
 'usb_speed': nan,
 'name': 'lexar lxqd32gcrbeu1400 - tarjeta de memoria de 32 gb',
 'price': nan,
 'brand': 'lexar',
 'size': '32 gb',
 'instance_id': 'altosight.com//5257',
 'real_cluster': 0}

In [319]:
key_li = list(truth_li[0].keys())
key_li.remove("instance_id")
key_li.remove("real_cluster")
key_li = ["real_cluster", "instance_id"] + key_li

In [320]:
with open("real.csv","w",encoding="utf8") as f:
    d = truth_li
    csv_writer = csv.DictWriter(f, key_li)
    csv_writer.writeheader()
    csv_writer.writerows(d)

In [321]:
len(truth_dict)

812

In [322]:
my_y = pd.read_csv(my_y_filename)

In [323]:
my_y_list = my_y.to_dict("records")

In [324]:
my_y_list[0]

{'Cluster ID': 0,
 'confidence_score': 0.9139723079284252,
 'type': nan,
 'uhs': nan,
 'line': 'xqd',
 'mymodel': '1400x',
 'micro': 'not micro',
 'video_speed': nan,
 'usb_speed': nan,
 'name': 'lexar 32gb 1400x 210mb/s professional xqd memoriakartya',
 'price': 31990.0,
 'brand': 'lexar',
 'size': '32 gb',
 'instance_id': 'altosight.com//0'}

In [325]:
for item in my_y_list:
    if item["instance_id"]=="source0__17878":
        print(item)

In [326]:
for item in my_y_list:
    if item["instance_id"] in truth_dict:
        item["real"] = truth_dict[item["instance_id"]]
    else:
        item["real"] = -1

In [327]:
iid_to_cluster_id = {}
iid_to_real_id = {}
real_id_iids = {}
cluster_id_iids = {}

for item in my_y_list:
    iid, cluster_id, real_id = item["instance_id"], item["Cluster ID"], item["real"]
    iid_to_cluster_id[iid] = cluster_id
    iid_to_real_id[iid] = real_id
    
    if real_id not in real_id_iids:
        real_id_iids[real_id] = []
    real_id_iids[real_id].append(iid)
    
    if cluster_id not in cluster_id_iids:
        cluster_id_iids[cluster_id] = []
    cluster_id_iids[cluster_id].append(iid)

In [328]:
from itertools import permutations, combinations

In [329]:
for item in my_y_list:
    iid, cluster_id, real_id = item["instance_id"], item["Cluster ID"], item["real"]
    
    if item["real"] != -1:
        #print(real_id_iids[item["real"]])
        total_count = 0
        total_found = 0
        for cb in combinations(real_id_iids[real_id], 2):
            total_count += 1
            if cb[0] +"|" + cb[1] in mine_dict or cb[1]+"|"+cb[0] in mine_dict:
                total_found += 1
                
        #print(total_count)
        #print(total_found)
        #print(total_count - total_found)
        item["r1_total"] = total_count
        item["r1_found"] = total_found
        item["r1_fn_missing"] = total_count - total_found
    else:
        item["r1_total"] = -1
        item["r1_found"] = -1
        item["r1_fn_missing"] = -1
        
    if cluster_id != -1:
        total_count = 0
        total_real = 0
        for cb in combinations(cluster_id_iids[cluster_id], 2):
            total_count += 1
            if cb[0]+"|"+cb[1] in gold_dict or cb[1]+"|"+cb[0] in gold_dict:
                total_real += 1
        item["r2_total"] = total_count
        item["r2_real"] = total_real
        item["r2_fp_extra"] = total_count - total_real
    else:
        item["r2_total"] = -1
        item["r2_real"] = -1
        item["r2_fp_extra"] = -1

    

    
    c1 = Counter([i["Cluster ID"] for i in my_y_list if i["real"]==item["real"]])
    c2 = Counter([i["real"] for i in my_y_list if i["Cluster ID"]==item["Cluster ID"]])
    
#     print(item)
    item["r1"] = str(dict(c1.most_common()))
    item["r2"] = str(dict(c2.most_common()))
    


In [330]:
list(c1)

[57]

In [331]:
key_li = list(item.keys())
key_li.remove("real")
key_li.remove("instance_id")
key_li.remove("Cluster ID")
key_li.remove("r1")
key_li.remove("r2")
for k in ["r1_total", "r1_found", "r1_fn_missing", "r2_total", "r2_real", "r2_fp_extra"]:
    key_li.remove(k)
key_li = ["Cluster ID","real","r1","r1_total", "r1_found", "r1_fn_missing","r2","r2_total", "r2_real", "r2_fp_extra","instance_id"] + key_li

In [332]:
with open(final_save_name,"w",encoding="utf8") as f:
    d = my_y_list
    csv_writer = csv.DictWriter(f, key_li)
    csv_writer.writeheader()
    csv_writer.writerows(d)

In [40]:
from collections import defaultdict
current_cluster_id = 0
record_to_cluster = {}
cluster_records = defaultdict(set)

In [43]:
for r in gold_dict:
    a, b = (r.split('|'))
    
    # Check the case
    if (a not in record_to_cluster and b not in record_to_cluster):
        # Create a new cluster
        record_to_cluster[a] = current_cluster_id
        record_to_cluster[b] = current_cluster_id
        cluster_records[current_cluster_id].update({a, b})
        current_cluster_id += 1
    elif(a in record_to_cluster and b not in record_to_cluster):
        c = record_to_cluster[a]
        record_to_cluster[b] = c
        cluster_records[c].update({b})
        pass
    elif(b in record_to_cluster and a not in record_to_cluster):
        c = record_to_cluster[b]
        record_to_cluster[a] = c
        cluster_records[c].update({a})
    elif (a in record_to_cluster and b in record_to_cluster and record_to_cluster[a] != record_to_cluster[b]):
        # merge two clusters
        # Get the cluster id of to be kept cluster
        c_keep = record_to_cluster[a]
        c_rem = record_to_cluster[b]
        record_to_cluster[b] = c_keep
        cluster_records[c_keep].update(cluster_records[c_rem])
        del cluster_records[c_rem]

In [45]:
len(cluster_records)

96